# Parsing EXIOBASE dataset files for the years 2010-2019

* To handle, aggregate and analyse this data, PYMRIO documentation (Stadler 2021) was used. 
* All EXIOBASE data files can be accessed and downloaded here: https://zenodo.org/records/5589597

## Importing all needed libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import pymrio

# Read and download 2019 data

In [21]:
# Using pymrio functions to parse the data
exio_19 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2019_pxp')
exio_19.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [22]:
# the following steps helps to extract the data for the case study - Germany and the information only for final government expenditure

gov_demand = exio_19.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

# drop all 0 values
gov_demand = gov_demand[gov_demand.gov_demand != 0]

# resetting the index. As we are working with multi-index tables, this step is necessary to make the analysis easier
de_cba = exio_19.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

# extracting production based accounts information
pba_without_de = exio_19.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_19.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

# merging the two tables: consumption based accounts and production based accounts
cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_19.Y.DE['Final consumption expenditure by government']

# access ghg emissions on F matrix and multiply by the government expenditure to get the total emissions of government expenditure per product
ghg_emissions_de = exio_19.L.dot(exio_19.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

# rename the columns to region, sector and gov_exp_ghg
ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']

# drop all the sectors with zero emissions
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

# add the satellite account to the analysis
de_cba_satellite = exio_19.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

# dropping the DE column from the satellite account not to have duplicated columns
exio_19.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_19.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})

# merging the two tables: consumption based accounts and production based accounts: including production in Germany that was then purchased by the government
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

# final merge of the tables for 2019
de_gov_all_19 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_19.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2067831034.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_19.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2067831034.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_19.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2165, 1241)

In [23]:
# export the data to a csv file
de_gov_all_19.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_19.csv', index=False)

# Read and download 2018 data

In [24]:
# repeat the same steps for the 2018 data
exio_18 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2018_pxp')
exio_18.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [25]:
# the following steps helps to extract the data for the case study - Germany and the information only for final government expenditure
gov_demand = exio_18.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

# drop all 0 values
gov_demand = gov_demand[gov_demand.gov_demand != 0]

# resetting the index. As we are working with multi-index tables, this step is necessary to make the analysis easier
de_cba = exio_18.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

# extracting production based accounts information
pba_without_de = exio_18.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_18.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

# merging the two tables: consumption based accounts and production based accounts
cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_18.Y.DE['Final consumption expenditure by government']

# access ghg emissions on F matrix and multiply by the government expenditure to get the total emissions of government expenditure per product
ghg_emissions_de = exio_18.L.dot(exio_18.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

# rename the columns to region, sector and gov_exp_ghg
ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']

# drop all the sectors with zero emissions
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

# add the satellite account to the analysis
de_cba_satellite = exio_18.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

# dropping the DE column from the satellite account not to have duplicated columns
exio_18.satellite.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de_satellite = exio_18.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})

# merging the two tables: consumption based accounts and production based accounts: including production in Germany that was then purchased by the government
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

# final merge of the tables for 2018
de_gov_all_18 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_18.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/3564263434.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_18.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/3564263434.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_18.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2089, 1241)

In [26]:
de_gov_all_18.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_18.csv', index=False)

# Read 2017 data

### The same steps are repeated for the following datasets

In [27]:
exio_17 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2017_pxp')
exio_17.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [28]:
gov_demand = exio_17.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})


gov_demand = gov_demand[gov_demand.gov_demand != 0]


de_cba = exio_17.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_17.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_17.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_17.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_17.L.dot(exio_17.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_17.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_17.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_17.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_17 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_17.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/3369811827.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_17.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/3369811827.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_17.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2327, 1241)

In [29]:
de_gov_all_17.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_17.csv', index=False)

# Read 2016 data

In [30]:
exio_16 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2016_pxp')
exio_16.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [31]:
gov_demand = exio_16.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_16.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_16.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_16.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_16.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_16.L.dot(exio_16.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']

ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_16.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_16.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_16.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_16 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_16.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/4031619153.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_16.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/4031619153.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_16.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2374, 1241)

In [32]:
de_gov_all_16.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_16.csv', index=False)

## Parsing 2015 data

In [33]:
exio_15 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2015_pxp')
exio_15.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [34]:
gov_demand = exio_15.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_15.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_15.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_15.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_15.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_15.L.dot(exio_15.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']

ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_15.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_15.satellite.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de_satellite = exio_15.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_15 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_15.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2002768507.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_15.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2002768507.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_15.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2158, 1241)

In [35]:
de_gov_all_15.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_15.csv', index=False)

# Parsing 2014 data

In [2]:
exio_14 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2014_pxp')
exio_14.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [3]:
gov_demand = exio_14.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_14.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_14.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_14.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_14.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_14.L.dot(exio_14.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_14.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_14.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_14.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_14 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_14.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/1575378226.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_14.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/1575378226.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_14.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2353, 1241)

In [4]:
de_gov_all_14.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_14.csv', index=False)

# Parsing 2013 data

In [5]:
exio_13 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2013_pxp')
exio_13.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [6]:
gov_demand = exio_13.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_13.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_13.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_13.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_13.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_13.L.dot(exio_13.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_13.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_13.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_13.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_13 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_13.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/754253248.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_13.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/754253248.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_13.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2182, 1241)

In [7]:
de_gov_all_13.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_13.csv', index=False)

# Parsing 2012 data

In [14]:
exio_12 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2012_pxp')
exio_12.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [15]:
gov_demand = exio_12.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_12.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_12.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_12.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_12.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_12.L.dot(exio_12.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_12.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_12.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_12.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_12 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_12.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2508339453.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_12.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/2508339453.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_12.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2360, 1241)

In [16]:
de_gov_all_12.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_12.csv', index=False)

# Parsing 2011

In [11]:
exio_11 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2011_pxp')
exio_11.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [12]:
gov_demand = exio_11.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_11.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_11.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_11.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)
gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_11.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_11.L.dot(exio_11.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_11.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_11.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_11.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_11 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_11.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/1686058180.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_11.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/1686058180.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_11.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2402, 1241)

In [13]:
de_gov_all_11.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_11.csv', index=False)

# Parsing 2010 data

In [18]:
exio_10 = pymrio.parse_exiobase3('/Users/gresasmolica/Extracted_Data/IOT_2010_pxp')
exio_10.calc_all()

/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:890: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  Y_agg = Y.groupby(level="region", axis=1, sort=False).sum()
/opt/homebrew/lib/python3.10/site-packages/pymrio/core/mriosystem.py:935: FutureWarning: DataFrame.groupby with axis=1 is deprecated. Do `frame.T.groupby(...)` without axis instead.
  F_Y_agg = self.F_Y.groupby(level="region", axis=1, sort=False).sum()


In [19]:
gov_demand = exio_10.Y.DE['Final consumption expenditure by government']
gov_demand = gov_demand.to_frame().reset_index()
gov_demand = gov_demand.rename(columns={'Final consumption expenditure by government': 'gov_demand'})

gov_demand = gov_demand[gov_demand.gov_demand != 0]

de_cba = exio_10.impacts.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba.insert(0, 'region', 'DE')

pba_without_de = exio_10.impacts.D_pba.drop('DE', axis=1, inplace=True)
pba_without_de = exio_10.impacts.D_pba.T
pba_without_de = pba_without_de.reset_index().rename(columns={'index': 'sector'})

cba_pba = pd.concat([pba_without_de, de_cba], ignore_index=True)

gov_demand_pba_cba = pd.merge(cba_pba, gov_demand, on=['sector', 'region'], how='inner')
gov_demand_pba_cba = gov_demand_pba_cba.loc[:, (gov_demand_pba_cba != 0).any(axis=0)]

de_gov = exio_10.Y.DE['Final consumption expenditure by government']

ghg_emissions_de = exio_10.L.dot(exio_10.impacts.F.loc["GHG emissions (GWP100) | Problem oriented approach: baseline (CML, 2001) | GWP100 (IPCC, 2007)"]).mul(de_gov, axis=0)
ghg_emissions_de = ghg_emissions_de.to_frame().reset_index()

ghg_emissions_de.columns = ['region', 'sector', 'gov_exp_ghg']
ghg_emissions_de = ghg_emissions_de[ghg_emissions_de['gov_exp_ghg'] != 0]

gov_demand_pba_cba_ghg = pd.merge(gov_demand_pba_cba, ghg_emissions_de, on=['sector', 'region'], how='inner')

de_cba_satellite = exio_10.satellite.D_cba.DE.T.reset_index().rename(columns={'index': 'category'})
de_cba_satellite.insert(0, 'region', 'DE')

exio_10.satellite.D_pba.drop('DE', axis=1, inplace=True)

pba_without_de_satellite = exio_10.satellite.D_pba.T.reset_index().rename(columns={'index': 'category'})
cba_pba_satellite = pd.concat([pba_without_de_satellite, de_cba_satellite], ignore_index=True)

de_gov_all_10 = pd.merge(cba_pba_satellite, gov_demand_pba_cba_ghg, on=['sector', 'region'], how='inner') 
de_gov_all_10.shape

/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/4189295213.py:14: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  pba_without_de = exio_10.impacts.D_pba.drop('DE', axis=1, inplace=True)
/var/folders/38/sb94bl795bv7klhlb8l7pj_00000gn/T/ipykernel_94251/4189295213.py:43: PerformanceWarning: dropping on a non-lexsorted multi-index without a level parameter may impact performance.
  exio_10.satellite.D_pba.drop('DE', axis=1, inplace=True)


(2333, 1241)

In [20]:
de_gov_all_10.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all_10.csv', index=False)

# Bringing all the datasets together

### Add a "year" column for all files

In [36]:
de_gov_all_10['year'] = 2010
de_gov_all_11['year'] = 2011
de_gov_all_12['year'] = 2012
de_gov_all_13['year'] = 2013
de_gov_all_14['year'] = 2014
de_gov_all_15['year'] = 2015
de_gov_all_16['year'] = 2016
de_gov_all_17['year'] = 2017
de_gov_all_18['year'] = 2018
de_gov_all_19['year'] = 2019

In [37]:
de_gov_all = pd.concat([de_gov_all_10, de_gov_all_11, de_gov_all_12, de_gov_all_13, de_gov_all_14, de_gov_all_15, de_gov_all_16, de_gov_all_17, de_gov_all_18, de_gov_all_19], ignore_index=True)

In [39]:
# export the data to a csv file
de_gov_all.to_csv('/Users/gresasmolica/Desktop/Gresa Smolica/Hertie - MDS/Master_thesis/de_gov_all.csv', index=False)